# 텍스트 마이닝 프로젝트

`(1)` 관심있는 분야의 데이터를 웹에서 크롤링하여 데이터 셋을 생성 (가급적 관측치는 1000개 이상의 데이터를 활용, 2~3개의 데이터 화일로 만들어 bind 하여 이용해도 됨)

`(2)` 사전처리를 수행하고 데이터를 분석

`(3)` 프로젝트 결과는 보고서 형태로 만들어 파일로 저장하여 업로드

`-` 크롤링, 사전처리, 데이터 분석 코드는 보고서 맨 뒷부분에 첨부

`-` 코드는 R 또는 Python 둘 중 하나로 작성

## 주제

`-` 사람로 사이트에 있는 명언 텍스트를 분석 (https://saramro.com)

## 크롤링

In [20]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from bs4.element import Tag
from tqdm.auto import tqdm

from typing import List

In [32]:
class SaramroCrawler:
    """Saramro(https://saramro.com/) 사이트를 크롤링"""
    _base_url = 'https://saramro.com'
    _content_exists_error_message_title = '오류안내 페이지 | 사람로'
    
    def __init__(self):
        self.quotes_url = f'{self._base_url}/quotes'
        self.quotes_storage = []

    def get_quotes(self, start_id: int, end_id: int) -> List[str]:
        for id in tqdm(range(start_id, end_id + 1)):
            quote = self.get_quote(id)
            if quote:
                self.quotes_storage.append(quote)
        return self.quotes_storage
    
    def get_quote(self, id: int) -> List[str]:
        url = f'{self.quotes_url}/{id}'
        response = requests.get(url)
        if response.status_code != 200:
            return []
        
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        if not self._check_post_exists(soup):
            return []
             
        title = self._get_title_from_soup(soup)
        content = self._get_content_from_soup(soup)
        
        try:
            quote = self._get_quote_from_content(content)
            author = self._get_author_from_content(content)
            categories = self._get_categories_from_title(title)
            return [quote, author, categories]
        except:
            return []
    
    def _check_post_exists(self, soup: BeautifulSoup) -> bool:
        post_title = soup.select_one('title').text
        if post_title == self._content_exists_error_message_title:
            return False
        return True
    
    def _get_title_from_soup(self, soup: BeautifulSoup) -> Tag:
        title = soup.select_one('#bo_v_title > span')
        return title
    
    def _get_content_from_soup(self, soup: BeautifulSoup) -> Tag:
        content = soup.select_one('#bo_v_con')
        return content
    
    def _get_quote_from_content(self, content: Tag) -> str:
        quote = content.contents[0].strip()
        return quote
    
    def _get_author_from_content(self, content: Tag) -> str:
        author = content.contents[2].split('-')[-1].strip()
        return author
    
    def _get_categories_from_title(self, title: Tag) -> str:
        categories = title.text.split('-')[0].strip()
        return categories
    
    def _quotes2df(self) -> None:
        self.df = pd.DataFrame(self.quotes_storage, columns=['quote', 'author', 'categories'])
    
    def save_quotes2csv(self, path: str) -> None:
        self._quotes2df()
        self.df.to_csv(path)

`-` 2022-12-08 17:08:00 기준 명언 모음 게시판에 명언글 아이디는 1부터 16402까지 존재한다

`-` 삭제된 게시글의 번호는 존재하지 않으므로 이를 제외하면 명언의 개수는 13387개이다 

`-` 최종적으로 13381개의 명언을 크롤링했다

In [33]:
start_id = 1
end_id = 16402
crawler = SaramroCrawler() 

In [28]:
quotes = crawler.get_quotes(start_id, end_id)

  0%|          | 0/16402 [00:00<?, ?it/s]

In [29]:
path = './data/quote.csv'
crawler.save_quotes2csv(path)